In [11]:
## imports 
import pandas as pd
import numpy as np
import yaml
import warnings
#warnings.filterwarnings("ignore", category=UserWarning)

## way to connect to mysql
## if you need to install
## uncomment this line:
#! pip install mysql-connector-python
import mysql.connector

## function to feed path name to load
## credentials
def load_creds(path: str):
    with open(path, 'r') as stream:
        try:
            creds = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    return(creds)

pd.options.display.max_rows = 999
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Preliminary: define connection and read sample of data

In [27]:
## read in creds; change the path name if stored
## elsewhere
creds = load_creds("../09_db_cred.yaml")

In [14]:
creds

{'practice_database': {'user': 'qss20',
  'password': 'xtPdDZHnxKMFSD5RyP',
  'port': 3306,
  'database': 'sentencing',
  'host': 'qss20-7570-db.c.dartmouth.edu'}}

In [15]:
creds['practice_database']['database']

'sentencing'

In [28]:
## connect to the database
cnx = mysql.connector.connect(
    user=creds['practice_database']['user'], 
    password=creds['practice_database']['password'],
    port=creds['practice_database']['port'],
    database= "sentencing",
    host = creds['practice_database']['host'])
cursor = cnx.cursor(buffered=True)
cnx

In [29]:
q = """
SELECT 
    *
FROM caseinit
LIMIT 5
"""
pd.read_sql_query(q, cnx)

/var/folders/0w/2wvbn9xx39scmwd7157fzlw80000gn/T/ipykernel_1143/977434783.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(q, cnx)


,CASE_ID,CASE_PARTICIPANT_ID,RECEIVED_DATE,OFFENSE_CATEGORY,PRIMARY_CHARGE_FLAG,CHARGE_ID,CHARGE_VERSION_ID,DISPOSITION_CHARGED_OFFENSE_TITLE,CHARGE_COUNT,DISPOSITION_DATE,...,INCIDENT_END_DATE,LAW_ENFORCEMENT_AGENCY,LAW_ENFORCEMENT_UNIT,ARREST_DATE,FELONY_REVIEW_DATE,FELONY_REVIEW_RESULT,ARRAIGNMENT_DATE,UPDATED_OFFENSE_CATEGORY,is_in_diversion,in_chicago
0,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242195814523,155656315869,FIRST DEGREE MURDER,2,12/17/2014 12:00:00 AM,...,,CHICAGO POLICE DEPT,,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion,False,None
1,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242198287388,131513547452,HOME INVASION,14,12/17/2014 12:00:00 AM,...,,CHICAGO POLICE DEPT,,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion,False,None
2,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242351605056,176626576281,FIRST DEGREE MURDER,4,12/17/2014 12:00:00 AM,...,,CHICAGO POLICE DEPT,,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion,False,None
3,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242352841488,176617824190,FIRST DEGREE MURDER,5,12/17/2014 12:00:00 AM,...,,CHICAGO POLICE DEPT,,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion,False,None
4,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242356550787,131238606761,HOME INVASION,13,12/17/2014 12:00:00 AM,...,,CHICAGO POLICE DEPT,,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion,False,None


In [31]:
q = """
SELECT 
    CASE_ID,
    CASE_PARTICIPANT_ID
FROM caseinit
WHERE AGE_AT_INCIDENT > 40
LIMIT 5
"""
pd.read_sql_query(q, cnx)

/var/folders/0w/2wvbn9xx39scmwd7157fzlw80000gn/T/ipykernel_1143/1817232844.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(q, cnx)


,CASE_ID,CASE_PARTICIPANT_ID
0,205411772533,106247326235
1,208597923533,114147409332
2,209306582865,122563667565
3,209754034523,117614073313
4,210275284632,117990102885


# Activity 1

1. Create a new column -- `in_chicago` when pulling from the `caseinit` table that takes on the value of "YES" if INCIDENT_CITY = Chicago; "NO" otherwise (which represents incidents in Cook County suburbs outside the city limits);  and pull the table. Use `crosstabs` to confirm that this worked
2. Repeat step 1 but also filter out blank strings (`INCIDENT_CITY` == "")
3. Use `where` to row filter to initiations in Chicago and use group by to find the count of cases diverted and not diverted (`is_in_diversion`); pull the table with those counts
4. Modify the query in step 3 to find the proportion of cases in chicago diverted (hint you made need to use case when in a subquery)
5. Modify the query in step 4 to find the proportion of cases in chicago versus cases not in chicago sent to diversion 


In [35]:
varcreate

,CASE_ID,CASE_PARTICIPANT_ID,RECEIVED_DATE,OFFENSE_CATEGORY,PRIMARY_CHARGE_FLAG,CHARGE_ID,CHARGE_VERSION_ID,DISPOSITION_CHARGED_OFFENSE_TITLE,CHARGE_COUNT,DISPOSITION_DATE,...,LAW_ENFORCEMENT_AGENCY,LAW_ENFORCEMENT_UNIT,ARREST_DATE,FELONY_REVIEW_DATE,FELONY_REVIEW_RESULT,ARRAIGNMENT_DATE,UPDATED_OFFENSE_CATEGORY,is_in_diversion,in_chicago,in_chicago
0,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242195814523,155656315869,FIRST DEGREE MURDER,2,12/17/2014 12:00:00 AM,...,CHICAGO POLICE DEPT,,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion,False,None,No
1,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242198287388,131513547452,HOME INVASION,14,12/17/2014 12:00:00 AM,...,CHICAGO POLICE DEPT,,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion,False,None,No
2,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242351605056,176626576281,FIRST DEGREE MURDER,4,12/17/2014 12:00:00 AM,...,CHICAGO POLICE DEPT,,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion,False,None,No
3,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242352841488,176617824190,FIRST DEGREE MURDER,5,12/17/2014 12:00:00 AM,...,CHICAGO POLICE DEPT,,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion,False,None,No
4,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242356550787,131238606761,HOME INVASION,13,12/17/2014 12:00:00 AM,...,CHICAGO POLICE DEPT,,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion,False,None,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272289,597320386308,452719800958,07/18/2022 12:00:00 AM,Retail Theft,True,11294724616133,1148300237684,RETAIL THEFT,1,09/02/2022 12:00:00 AM,...,CHICAGO PD,,07/18/2022 09:20:00 AM,07/18/2022 12:00:00 AM,Approved,09/02/2022 12:00:00 AM,Retail Theft,True,None,Yes
272290,597328542979,452727325242,07/18/2022 12:00:00 AM,Aggravated Fleeing and Eluding,True,11295052270826,1148334245809,AGGRAVATED FLEEING OR ATTEMPT TO ELUDE A PEACE...,1,09/06/2022 12:00:00 AM,...,EVERGREEN PARK PD,,07/18/2022 11:11:00 PM,07/18/2022 12:00:00 AM,Approved,09/06/2022 12:00:00 AM,Aggravated Fleeing and Eluding,False,None,No
272291,597341166398,452739650297,07/19/2022 12:00:00 AM,Criminal Damage to Property,True,11295731072434,1148405887926,CRIMINAL DAMAGE TO GOVERNMENT SUPPORTED PROPERTY,1,08/23/2022 12:00:00 AM,...,COOK COUNTY SHERIFF (IL0160000),,,08/02/2022 12:00:00 AM,Approved,08/17/2022 12:00:00 AM,Criminal Damage to Property,False,None,Yes
272292,597341166398,452739650297,07/19/2022 12:00:00 AM,Criminal Damage to Property,True,11295731072434,1148405887926,CRIMINAL DAMAGE TO GOVERNMENT SUPPORTED PROPERTY,1,08/23/2022 12:00:00 AM,...,COOK COUNTY SHERIFF (IL0160000),,,08/02/2022 12:00:00 AM,Approved,08/17/2022 12:00:00 AM,Criminal Damage to Property,False,None,Yes


In [39]:
%%time
## step 1
query_createvar = """
SELECT 
    *,
    CASE 
        when INCIDENT_CITY = 'Chicago' then 'Yes'
        ELSE 'No'
    END AS in_chicago_1
FROM caseinit
"""

varcreate = pd.read_sql_query(query_createvar, cnx)
pd.crosstab(varcreate.in_chicago_1, varcreate.INCIDENT_CITY).T.head()
pd.crosstab(varcreate.in_chicago_1, varcreate.INCIDENT_CITY).T.tail()

<timed exec>:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.


CPU times: user 22.5 s, sys: 2.69 s, total: 25.2 s
Wall time: 37.3 s


in_chicago_1,No,Yes
INCIDENT_CITY,,
Wilmette,333,0
Winnetka,129,0
Wood Dale,3,0
Woodstock,1,0
Worth,284,0


In [41]:
## step 2
query_createvar_noblank = """
SELECT 
    INCIDENT_CITY,
    CASE 
        when INCIDENT_CITY = 'Chicago' then 'Yes'
        ELSE 'No'
    END AS in_chicago_1
FROM caseinit
WHERE INCIDENT_CITY <> ""
"""

varcreate_nb = pd.read_sql_query(query_createvar_noblank, cnx)
pd.crosstab(varcreate_nb.in_chicago_1, varcreate_nb.INCIDENT_CITY).T.head(20)
pd.crosstab(varcreate_nb.in_chicago_1, varcreate_nb.INCIDENT_CITY).T.tail(15)


/var/folders/0w/2wvbn9xx39scmwd7157fzlw80000gn/T/ipykernel_1143/3004822840.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  varcreate_nb = pd.read_sql_query(query_createvar_noblank, cnx)


in_chicago_1,No,Yes
INCIDENT_CITY,,
Addison,2,0
Albers,1,0
Algonquin,3,0
Alsip,568,0
Antioch,2,0
Arlington Heights,1291,0
Aurora,3,0
Barrington,136,0
Barrington Hills,16,0


in_chicago_1,No,Yes
INCIDENT_CITY,,
Waukegan,5,0
West Chicago,1,0
West Dundee,1,0
Westchester,341,0
Western Springs,52,0
Westmont,1,0
Wheaton,4,0
Wheeling,1094,0
Willow Springs,82,0


In [ ]:
df.groupby("is_in_diversion").count()

In [42]:
## step 3
query_countdiv = """
SELECT 
    count(*) as count_div, 
    is_in_diversion
FROM caseinit
WHERE INCIDENT_CITY = 'Chicago'
GROUP BY is_in_diversion
"""
count_div = pd.read_sql_query(query_countdiv, cnx)
count_div

/var/folders/0w/2wvbn9xx39scmwd7157fzlw80000gn/T/ipykernel_1143/79265323.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  count_div = pd.read_sql_query(query_countdiv, cnx)


,count_div,is_in_diversion
0,167171,False
1,6402,True


In [43]:
## step 4
query_propdiv = """
SELECT 
    avg(is_div_bool) as prop_div
FROM  (
    SELECT 
        *,
        CASE 
            WHEN is_in_diversion = 'True' THEN 1
            ELSE 0
        END is_div_bool
    FROM caseinit) AS d
WHERE INCIDENT_CITY = 'Chicago'
"""
prop_div = pd.read_sql_query(query_propdiv, cnx)
prop_div

/var/folders/0w/2wvbn9xx39scmwd7157fzlw80000gn/T/ipykernel_1143/431508457.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  prop_div = pd.read_sql_query(query_propdiv, cnx)


,prop_div
0,0.0369


In [47]:
## step 5
query_propdiv_compare = """
SELECT 
    AVG(is_div_bool) as prop_div, 
    in_chicago_1
FROM  (
    SELECT 
        *,
        CASE 
            WHEN is_in_diversion = 'True' THEN 1
            ELSE 0
        END as is_div_bool,
        CASE 
            WHEN INCIDENT_CITY = 'Chicago' THEN 'Yes'
            ELSE 'No'
        END AS in_chicago_1
    FROM caseinit) AS d
WHERE INCIDENT_CITY <> ''
GROUP BY in_chicago_1
"""
prop_div_compare = pd.read_sql_query(query_propdiv_compare, cnx)
prop_div_compare

/var/folders/0w/2wvbn9xx39scmwd7157fzlw80000gn/T/ipykernel_1143/80468566.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  prop_div_compare = pd.read_sql_query(query_propdiv_compare, cnx)


,prop_div,in_chicago_1
0,0.0470,No
1,0.0369,Yes


# Activity 2 

1. Use the following crosswalk and the `CASE` variable in the `divert` table to create a new variable `DIVERSION_PROGRAM_TEXT` that spells out the diversion programs

DC: Drug Court

DDPP: Drug Deferred Prosecution

DS: Drug School

RJCC: Restorative Justice

MHC: Mental Health Court

VC: Veteran Court

2. Build on the query from step 1 to filter to Narcotics as the `UPDATED_OFFENSE_CATEGORY` and Black or White defendants (based on race in the diversions table) (hint: you'll need to join with the caseinit table based on case_id and case_participant_id, you can do a inner join to keep only those diverted). Select the case_id, case_participant_id, case, race, and diversion_program_text columns

3. Built on the query from step 2 (and/or modify to just focus on drug school and drug court) to find the (1) rate of Black defendants sent to drug court, (2) rate of white defendants sent to drug court, (3) rate of Black defendants sent to drug school, and (4) rate of white defendants sent to drug school


In [38]:
## step 1; query creating var
dp_type = """
SELECT *,
    CASE WHEN DIVERSION_PROGRAM = 'DC' THEN 'Drug Court'
    WHEN DIVERSION_PROGRAM = 'DDPP' THEN 'Drug Deferred Prosecution'
    WHEN DIVERSION_PROGRAM = 'DS' THEN 'Drug School'
    WHEN DIVERSION_PROGRAM = 'RJCC' THEN 'Restorative Justice'
    WHEN DIVERSION_PROGRAM = 'MHC' THEN 'Mental Health Court'
    WHEN DIVERSION_PROGRAM = 'VC' THEN 'Veteran Court'
ELSE 'Other'
END as DIVERSION_PROGRAM_TEXT
FROM divert
"""

In [39]:
n = pd.read_sql_query(dp_type, cnx)
pd.crosstab(n.DIVERSION_PROGRAM,
           n.DIVERSION_PROGRAM_TEXT)

/var/folders/0w/2wvbn9xx39scmwd7157fzlw80000gn/T/ipykernel_38920/1975159586.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.


DIVERSION_PROGRAM_TEXT,Drug Court,Drug Deferred Prosecution,Drug School,Mental Health Court,Other,Restorative Justice,Veteran Court
DIVERSION_PROGRAM,,,,,,,
ACT,0,0,0,0,668,0,0
ARI,0,0,0,0,879,0,0
BR9,0,0,0,0,5542,0,0
DC,3530,0,0,0,0,0,0
DDPP,0,5686,0,0,0,0,0
DS,0,0,5705,0,0,0,0
MHC,0,0,0,2188,0,0,0
RJCC,0,0,0,0,0,386,0
SEED,0,0,0,0,166,0,0


In [26]:
## step 2: add the filters 
dp_type_filtered = """
SELECT 
    c.CASE_ID, 
    c.CASE_PARTICIPANT_ID,
    d.RACE, 
    DIVERSION_PROGRAM_TEXT
FROM  (
    SELECT 
        *,
        CASE 
            WHEN DIVERSION_PROGRAM = 'DC' THEN 'Drug Court'
            WHEN DIVERSION_PROGRAM = 'DDPP' THEN 'Drug Deferred Prosecution'
            WHEN DIVERSION_PROGRAM = 'DS' THEN 'Drug School'
            WHEN DIVERSION_PROGRAM = 'RJCC' THEN 'Restorative Justice'
            WHEN DIVERSION_PROGRAM = 'MHC' THEN 'Mental Health Court'
            WHEN DIVERSION_PROGRAM = 'VC' THEN 'Veteran Court'
            ELSE 'Other'
        END as DIVERSION_PROGRAM_TEXT
    FROM divert) AS d
INNER JOIN (
    SELECT 
        UPDATED_OFFENSE_CATEGORY,
        CASE_ID, 
        CASE_PARTICIPANT_ID
    FROM caseinit
    ) AS c
ON 
    c.CASE_ID = d.CASE_ID AND
    c.CASE_PARTICIPANT_ID = d.CASE_PARTICIPANT_ID
WHERE d.RACE IN ("Black", "White")
AND UPDATED_OFFENSE_CATEGORY = 'Narcotics'
"""

In [27]:
d_filtered = pd.read_sql_query(dp_type_filtered, cnx)
d_filtered.head()


/var/folders/0w/2wvbn9xx39scmwd7157fzlw80000gn/T/ipykernel_38920/3717909997.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.


,CASE_ID,CASE_PARTICIPANT_ID,RACE,DIVERSION_PROGRAM_TEXT
0,506071709491,347654987555,White,Mental Health Court
1,506071709491,347654987555,White,Mental Health Court
2,506106860859,347693763008,Black,Drug School
3,506131913491,347722752275,White,Drug Court
4,506131913491,347722752275,White,Drug Court
